In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "mps" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

prompt = "问题：胃癌最主要的转移途径是（　　）。\n选项：\n\tA. 直接蔓延\n\tB. 淋巴转移\n\tC. 血行转移\n\tD. 盆腔转移"
messages = [
    {"role": "system", "content": "假设您是一名医生，请根据患者的症状回答以下选择题。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Can't infer missing attention mask on `mps` device. Please provide an `attention_mask` or use a different device.